# EDA Datario
Ao invés de começar por diversas queries dentro do GCP, foi decidido explorar os dados primeiramente através do Pandas, visando decidir todas as variáveis e features necessárias para responder todas as questões do desafio e também ter insights sobre possíveis análises extras.
<br>Objetivos principais:
- criar arquivos .csv para não consumir dados do GCP a cada vez que o código for rodado
- ter uma visão geral de cada dataset
- Decidir quais colunas são necessárias para a soluções de **Todas** as questões do desafio
- Analisar a possibilidade de análises extras relevantes para os dashboards PowerBI e Streamlit.

## Perguntas SQL

## Localização de chamados do 1746
#### Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.

1. Quantos chamados foram abertos no dia 01/04/2023?
2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?


## Chamados do 1746 em grandes eventos
#### Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego".

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
8. Quantos chamados desse subtipo foram abertos em cada evento?
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

##### Importante: a tabela de Chamados do 1746 possui mais de 10M de linhas. Evite fazer consultas exploratórias na tabela sem um filtro ou limite de linhas para economizar sua cota no BigQuery!

## Imports

In [1]:
import basedosdados as bd
import pandas as pd
from datetime import timedelta
from utils import Weather

In [2]:
# Config
wrio = Weather(22.5423, 43.1021) # Cria instância da classe Weather com as coordenadas da cidade do Rio de Janeiro

## Datasets
Conforme primeiras observações feitas no notebook 'api_first_test.ipynb'. Vamos começar a analisar as tabelas do BigQuery em ordem da menor para a maior.
<br>
- datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos
- datario.adm_central_atendimento_1746.chamado
- datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos

### Fluxo de visitantes/Ocupação de eventos 

Os códigos abaixo estão comentados para evitar o consumo desnecessário de dados no GCP

In [7]:
# Rodar a query apenas uma vez e salvar o dataframe como csv

#query_ocup = "SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos"
#ocup = bd.read_sql(query_ocup, billing_project_id="dados-rio-433111")
#ocup.to_csv('ocupacao.csv', index=False)

Downloading: 100%|██████████|


',ano,data_inicial,data_final,evento,taxa_ocupacao\r\n0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554\r\n1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251\r\n2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184\r\n3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451\r\n'

In [3]:
# Dataset Ocupação Original
ocup = pd.read_csv('ocupacao.csv')
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


Podemos observar que as observações foram feitas dentro do curto período de setembro/2022 até fevereiro/2023.
<br>Todos os eventos aconteceram em pelo menos 3 dias
<br>O Rock in Rio aconteceu por dois finais de semana seguidos, o ultimo tendo um fluxo de visitantes relativamente maior que o primeiro
<br>O evento de maior fluxo foi o Carnaval, provavelmente por conta de ser feriado público e possívelmente ter sido em dias de calor(verificar com api)

In [4]:
# Info do dataframe original
ocup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ano            4 non-null      object 
 1   data_inicial   4 non-null      object 
 2   data_final     4 non-null      object 
 3   evento         4 non-null      object 
 4   taxa_ocupacao  4 non-null      float64
dtypes: float64(1), object(4)
memory usage: 292.0+ bytes


**Tratativas**

In [10]:
# Converter colunas de datas para datetime
ocup['data_inicial'] = pd.to_datetime(ocup['data_inicial'])
ocup['data_final'] = pd.to_datetime(ocup['data_final'])
# Extrair apenas o ano(s) da coluna ano
ocup['ano'] = ocup['ano'].apply(lambda x: x.split()[-1])
# Criar colunas para o dia da semana de início e fim dos eventos escrita em Português
dias = {
    0: "Segunda-feira",
    1: "Terça-feira",
    2: "Quarta-feira",
    3: "Quinta-feira",
    4: "Sexta-feira",
    5: "Sábado",
    6: "Domingo"
}
ocup['dia_inicial'] = ocup['data_inicial'].apply(lambda x: x.weekday()).map(dias)
ocup['dia_final'] = ocup['data_final'].apply(lambda x: x.weekday()).map(dias)
# Coluna de duração dos eventos(quantidade de dias)
ocup['duracao'] = (ocup['data_final'] - ocup['data_inicial']) + timedelta(days=1)
# Temperatura/clima do dia utilizando o método forecast da classe Weather
ocup['temperatura_data_inicial'] = ocup['data_inicial'].apply(lambda x: wrio.forecast(x.date(), x.date())['temperatura media'])
ocup['clima_data_inicial'] = ocup['data_inicial'].apply(lambda x: wrio.forecast(x.date(), x.date())['clima'])
ocup['temperatura_data_final'] = ocup['data_final'].apply(lambda x: wrio.forecast(x.date(), x.date())['temperatura media'])
ocup['clima_data_final'] = ocup['data_final'].apply(lambda x: wrio.forecast(x.date(), x.date())['clima'])
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao,dia_inicial,dia_final,duracao,temperatura_data_inicial,clima_data_inicial,temperatura_data_final,clima_data_final
0,2023,2023-02-18,2023-02-21,Carnaval,0.9554,Sábado,Terça-feira,4 days,15.151581,Mainly Sunny,20.324499,Mainly Sunny
1,(2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,Sexta-feira,Domingo,3 days,10.391168,Rain,13.837001,Light Rain
2,2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,Sexta-feira,Domingo,3 days,30.980749,Sunny,32.174496,Sunny
3,2022,2022-09-08,2022-09-11,Rock in Rio,0.9451,Quinta-feira,Domingo,4 days,32.207832,Sunny,32.826580,Mainly Sunny


**Conclusão**
<br>Apesar de ter sido em época de chuva e temperaturas baixas, o Reveillon teve uma taxa de ocupação mais alta do que no primeiro final de semana do evento Rock in Rio
<br>O carnaval liderou a taxa de ocupação mesmo não sendo um dia considerado "Quente" para o cidadão Carioca

**Sugestão de análise futura**
A taxa de ocupação tem alguma relação com a quantidade de chamados de Perturbação do Sossego nas datas dos eventos?